## Configuraciones iniciales.

In [ ]:
import pandas as pd
import numpy as np
import joblib
import os
import psutil
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity


### Carga de datos.

In [7]:
df = pd.read_parquet(r'D:\Henry Data Science\Proyecto_individual_1\Data\modelo_dataset.parquet')

### Función para monitorear el uso de memoria.

In [8]:
def get_memory_usage():
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    return mem_info.rss / (1024 * 1024)  # Convertir de bytes a MB

### Configuración de TF - IDF con revisión de parámetros.

In [9]:
vectorizer = TfidfVectorizer(
    min_df=4, max_df=0.85, ngram_range=(1, 2), max_features=40000, dtype=np.float32
)

matriz = vectorizer.fit_transform(df['predictor'])

### Reducción de dimensionalidad y análisis de varianza explicada.

In [10]:
svd = TruncatedSVD(n_components=400, random_state=42)
matriz_reducida = svd.fit_transform(matriz)
explained_variance = svd.explained_variance_ratio_.sum()
print(f"Varianza explicada con SVD: {explained_variance:.2%}")

print(f"Uso de memoria: {get_memory_usage()} MB")

Varianza explicada con SVD: 15.23%
Uso de memoria: 520.13671875 MB


### Función de recomendación.

In [15]:
def limpiar_texto(texto):
    """Normaliza texto eliminando caracteres especiales y espacios redundantes."""
    texto = texto.lower()  # Convertir a minúsculas
    texto = re.sub(r"[^a-zA-Z0-9\s]", "", texto)  # Eliminar caracteres especiales
    return texto.strip()

def encontrar_coincidencia(titulo, titulos, umbral=0.75):
    """
    Encuentra el título más parecido en la lista basada en longitud común y similitud.
    """
    titulo = limpiar_texto(titulo)
    mejores_resultados = []
    for t in titulos:
        t_limpio = limpiar_texto(t)
        # Similaridad basada en longitud común
        coincidencia = len(set(titulo) & set(t_limpio)) / max(len(titulo), len(t_limpio))
        mejores_resultados.append((t, coincidencia))

    # Ordenar resultados por similitud
    mejores_resultados = sorted(mejores_resultados, key=lambda x: x[1], reverse=True)
    
    # Retornar el mejor si cumple con el umbral
    if mejores_resultados[0][1] >= umbral:
        return mejores_resultados[0][0]
    else:
        return None

def recomendacion(title, top_n=5):
    # Buscar coincidencia en títulos
    match = encontrar_coincidencia(title, df['title'])
    if not match:
        return {"Error": "Título no encontrado"}

    # Obtener índice del título encontrado
    idx_list = df.index[df['title'] == match].tolist()
    if not idx_list:
        return {"Error": "Título no encontrado"}
    idx = idx_list[0]

    # Calcular similitud de coseno
    sim_scores = cosine_similarity(matriz_reducida[idx].reshape(1, -1), matriz_reducida)
    sim_scores = list(enumerate(sim_scores[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Ajustar top_n al número de películas disponibles
    top_n = min(top_n, len(sim_scores) - 1)
    movie_indices = [i[0] for i in sim_scores[1 : top_n + 1]]

    # Generar recomendaciones
    recomendaciones = df['title'].iloc[movie_indices].tolist()
    return {"Recomendaciones": recomendaciones}

# Ejemplo de uso
print(recomendacion('toy story'))

NameError: name 're' is not defined

### Guardar los modelos optimizados.

In [13]:
directory = '../data/processed/'
if not os.path.exists(directory):
    os.makedirs(directory)

joblib.dump(vectorizer, os.path.join(directory, 'vectorizer.pkl'), compress=3)
joblib.dump(matriz_reducida, os.path.join(directory, 'matriz_reducida.pkl'), compress=3)

['../data/processed/matriz_reducida.pkl']

### Obtener tamaño de archivos comprimidos.

In [14]:
def get_file_size(file_path):
    try:
        size = os.path.getsize(file_path)
        return size / (1024 * 1024)
    except FileNotFoundError:
        return "El archivo no existe."

vectorizer_path = '../data/processed/vectorizer.pkl'
matriz_reducida_path = '../data/processed/matriz_reducida.pkl'

print(f"Tamaño del vectorizer: {get_file_size(vectorizer_path):.2f} MB")
print(f"Tamaño de la matriz reducida: {get_file_size(matriz_reducida_path):.2f} MB")


Tamaño del vectorizer: 0.44 MB
Tamaño de la matriz reducida: 31.63 MB
